In [1]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

traffic_sign8499


In [2]:
# 查看个人持久化工作区文件
# !ls /home/kesci/work/

In [3]:
# 查看当前kernerl下的package
# !pip list --format=columns

In [4]:
# 显示cell运行时长
%load_ext klab-autotime

# **一、分析数据集特点**

首先观察数据，看看要识别的交通标志种类多少（62），以及每类图像有多少。

In [5]:
!ls /home/kesci/input/traffic_sign8499/traffic-sign/train

00000  00006  00012  00018  00024  00030  00036  00042	00048  00054  00060
00001  00007  00013  00019  00025  00031  00037  00043	00049  00055  00061
00002  00008  00014  00020  00026  00032  00038  00044	00050  00056
00003  00009  00015  00021  00027  00033  00039  00045	00051  00057
00004  00010  00016  00022  00028  00034  00040  00046	00052  00058
00005  00011  00017  00023  00029  00035  00041  00047	00053  00059
time: 400 ms


In [6]:
!ls /home/kesci/input/traffic_sign8499/traffic-sign/test

00000  00006  00012  00018  00024  00030  00036  00042	00048  00054  00060
00001  00007  00013  00019  00025  00031  00037  00043	00049  00055  00061
00002  00008  00014  00020  00026  00032  00038  00044	00050  00056
00003  00009  00015  00021  00027  00033  00039  00045	00051  00057
00004  00010  00016  00022  00028  00034  00040  00046	00052  00058
00005  00011  00017  00023  00029  00035  00041  00047	00053  00059
time: 398 ms


In [7]:
from glob import glob
train_folds = glob('/home/kesci/input/traffic_sign8499/traffic-sign/train/*')
test_folds = glob('/home/kesci/input/traffic_sign8499/traffic-sign/test/*')
len(train_folds), len(test_folds)

(62, 62)

time: 9.78 ms


查看每个类别中图像的数量，以了解类别图像的均衡性。

In [8]:
len_train_folds = []
len_test_folds = []
for fold in train_folds:
    fold_files = glob(fold+'/*.png')
    len_train_folds.append(len(fold_files))
for fold in test_folds:
    fold_files = glob(fold+'/*.png')
    len_test_folds.append(len(fold_files))
# len(len_train_folds), len(len_test_folds)


time: 26.3 ms


In [9]:
# train_folds[0].split('/')[7]
# !pip install pyecharts
''' 不堆叠排列 '''
# import pyecharts
labels = [int(train_folds[idx].split('/')[7])  for idx in range(len(train_folds)) ]
# labels
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.subplot(2,1,1)
plt.bar(labels, len_train_folds, label='number of each train fold')
plt.legend()
plt.subplot(2,1,2)
plt.bar(labels, len_test_folds, label='number of each test fold')
plt.legend()
plt.show()

<Figure size 1200x600 with 2 Axes>

time: 827 ms


从上图可以看出，类别多包含62种，且每个类别的数目差异大、不均衡。总体特点，多类少样本。

# **二、搭建CNN模型分类**

用深度学习做图片分类考虑CNN模型，主流CNN模型多，在实验之前，难以知道哪个模型效果好。因此，先选择一个简单又经典的模型看一下分类效果，首先选用LeNet实现一下。

In [10]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
 
class LeNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model
        model = Sequential()
        inputShape = (height, width, depth)
        # if we are using "channels last", update the input shape
        if K.image_data_format() == "channels_first":   #for tensorflow
            inputShape = (depth, height, width)
        # first set of CONV => RELU => POOL layers
        model.add(Conv2D(20, (5, 5),padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        #second set of CONV => RELU => POOL layers
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model

Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


time: 14.8 s


参数解析器和一些参数的初始化

In [11]:
!pip install imutils

  Running setup.py bdist_wheel for imutils ... done
  Stored in directory: /home/kesci/.cache/pip/wheels/16/84/1f/bf88641293cda2c8be81a5c4b8ca973dd9125a6dc3767417fd
Successfully built imutils
You are using pip version 9.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
time: 20.7 s


In [12]:
'''
https://www.cnblogs.com/skyfsm/p/8051705.html
'''
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os
import sys
sys.path.append('..')
# from net.lenet import LeNet



def args_parse():
    # construct the argument parse and parse the arguments
    ap = argparse.ArgumentParser()
    ap.add_argument("-dtest", "--dataset_test", required=True,
        help="path to input dataset_test")
    ap.add_argument("-dtrain", "--dataset_train", required=True,
        help="path to input dataset_train")
    ap.add_argument("-m", "--model", required=True,
        help="path to output model")
    ap.add_argument("-p", "--plot", type=str, default="plot.png",
        help="path to output accuracy/loss plot")
    args = vars(ap.parse_args()) 
    return args

time: 3.81 s


In [13]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 35
INIT_LR = 1e-3
BS = 32
CLASS_NUM = 62
norm_size = 32

time: 596 µs


In [14]:
from PIL import Image
import numpy as np
import cv2
from keras.preprocessing.image import img_to_array

image = cv2.imread(fold_files[0])

# np.array(Image.open(fold_files[0])).shape, image.shape
pilimage = np.array(Image.open(fold_files[0]))
image0 = cv2.resize(image, (32, 32))
image = img_to_array(image0)
# image.shape
plt.subplot(2,1,1)
plt.imshow(pilimage)
plt.subplot(2,1,2)
plt.imshow(image0)
# image-image0

<Figure size 432x288 with 2 Axes>

time: 696 ms


# **三、载入数据**
读入数据，对读入的数据进行处理，并读入相应的标签信息。

In [15]:
def load_data(path):
    print('[INFO] loading images...')
    data = []
    labels = []
    # grab the image paths and randomly shuffle them
    imagePaths = sorted(list(paths.list_images(path)))
    random.seed(42)
    random.shuffle(imagePaths)
    # loop over the input images
    for imagePath in imagePaths:
        # load the image.preprocess it and sotre it in the data list
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (norm_size, norm_size))
        image = img_to_array(image)
        data.append(image)
        
        # extract the class label from the image path and update the labels list
        label = int(imagePath.split(os.path.sep)[-2])
        labels.append(label)
    
    # scale the raw pixel intensities to the image [0,1]
    data = np.array(data, dtype='float')/255.0
    labels = np.array(labels)
    
    # convert the labels from integers to vectores
    labels = to_categorical(labels, num_classes=CLASS_NUM)
    return data,labels

time: 1.82 ms


训练模型

In [16]:
# def train(aug, trainX, trainY, testX, testY, args):
#     # initialize the model
#     print('[INFO] compiling model....')
#     model = LeNet.build(width=norm_size, height=norm_size, depth=3
#     ,classes=CLASS_NUM)
#     opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)
#     model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
#     # train the network
#     print('[INFO] training network...')
#     H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
#     validation_data=(trainX, trainY), steps_per_epoch=len*(trainX)//BS,
#     epochs=EPOCHS, verbose=1)
#     # save the model to disk
#     print(['[INFO] serializing network...'])
#     model.save(args['model'])
#     # plot the training loss and accuracy
#     plt.style.use('ggplot')
#     plt.figure()
#     N = EPOCHS
#     plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
#     plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
#     plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
#     plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
#     plt.title("Training Loss and Accuracy on traffic-sign classifier")
#     plt.xlabel("Epoch #")
#     plt.ylabel("Loss/Accuracy")
#     plt.legend(loc="lower left")
#     plt.savefig(args["plot"])
    

time: 1.83 ms


In [17]:
def train(aug,trainX,trainY,testX,testY,args):
    # initialize the model
    print("[INFO] compiling model...")
    model = LeNet.build(width=norm_size, height=norm_size, depth=3, classes=CLASS_NUM)
    opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    model.compile(loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])

    # train the network
    print("[INFO] training network...")
    H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
        validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
        epochs=EPOCHS, verbose=1)

    # save the model to disk
    print("[INFO] serializing network...")
    model.save(args["model"])
    
    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    N = EPOCHS
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
    plt.title("Training Loss and Accuracy on traffic-sign classifier")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    # plt.savefig(args["plot"])

time: 2.93 ms


# **四、主函数**
在正式训练之前我们还使用了数据增广技术（ImageDataGenerator）来对我们的小数据集进行数据增强（对数据集图像进行随机旋转、移动、翻转、剪切等），以加强模型的泛化能力。

训练代码已经写好了，接下来开始训练（图片归一化尺寸为32，batch_size为32，epoches为35）。

In [18]:
#python train.py --dataset_train ../../traffic-sign/train --dataset_test ../../traffic-sign/test --model traffic_sign.model
if __name__=='__main__':
    # args = args_parse()
    # train_file_path = args["dataset_train"]
    train_file_path = '/home/kesci/input/traffic_sign8499/traffic-sign/train'
    # test_file_path = args["dataset_test"]
    test_file_path = '/home/kesci/input/traffic_sign8499/traffic-sign/test'
    args = {}
    args['model'] = 'traffic_sign_v1.model'
    trainX,trainY = load_data(train_file_path)
    testX,testY = load_data(test_file_path)
    # construct the image generator for data augmentation
    aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
        horizontal_flip=True, fill_mode="nearest")
    train(aug,trainX,trainY,testX,testY,args)

[INFO] loading images...
[INFO] loading images...
[INFO] compiling model...
[INFO] training network...
Epoch 1/35
142/142 [==============================] - 34s 241ms/step - loss: 2.6403 - acc: 0.3545 - val_loss: 1.7040 - val_acc: 0.5290
Epoch 2/35
142/142 [==============================] - 32s 227ms/step - loss: 1.5016 - acc: 0.5796 - val_loss: 1.0115 - val_acc: 0.6937
Epoch 3/35
142/142 [==============================] - 32s 227ms/step - loss: 1.0556 - acc: 0.6932 - val_loss: 0.7800 - val_acc: 0.7544
Epoch 4/35
142/142 [==============================] - 32s 227ms/step - loss: 0.8719 - acc: 0.7325 - val_loss: 0.5763 - val_acc: 0.8238
Epoch 5/35
142/142 [==============================] - 32s 227ms/step - loss: 0.7279 - acc: 0.7736 - val_loss: 0.4912 - val_acc: 0.8508
Epoch 6/35
142/142 [==============================] - 32s 228ms/step - loss: 0.6170 - acc: 0.8086 - val_loss: 0.6251 - val_acc: 0.7972
Epoch 7/35
142/142 [==============================] - 32s 228ms/step - loss: 0.5595 - a

<Figure size 432x288 with 1 Axes>

time: 19min 5s


# **五、预测单张图片
现在我们已经得到了我们训练好的模型traffic_sign.model，然后我们编写一个专门用于预测的脚本predict.py。
预测脚本中的代码编写思路是：参数解析器-》载入训练好的模型-》读入图片信息-》预测-》展示预测效果。值得注意的是，参数-s是用于可视化结果的，加上他的话我们就可以看出我们输入的图片以及模型预测的分类结果，很直观。如果只需要得到分类结果，不加-s就可以了。

In [19]:
# # import the necessary packages
# from keras.preprocessing.image import img_to_array
# from keras.models import load_model
# import numpy as np
# import argparse
# import imutils
# import cv2

# norm_size = 32

# def args_parse():
# # construct the argument parse and parse the arguments
#     ap = argparse.ArgumentParser()
#     ap.add_argument("-m", "--model", required=True,
#         help="path to trained model model")
#     ap.add_argument("-i", "--image", required=True,
#         help="path to input image")
#     ap.add_argument("-s", "--show", action="store_true",
#         help="show predict image",default=False)
#     args = vars(ap.parse_args())    
#     return args

# def predict(args):
#     print('[INFO] loading network...')
#     model = load_model(args['model'])
    
#     # load the iamge
#     image = cv2.imread(args['image'])
#     orig = image.copy()
    
#     # pre-processing the image for classifcation
#     image = cv2.resize(image, (norm_size, norm_size))
#     image = image.astype('float')/255.0
#     image = img_to_array(image)
#     image = np.expand_dims(image, axis=0)
    
#     # classify the input image
#     result = model.predict(image)[0]
    
#     proba = np.max(result)
#     label = str(np.where(result==proba)[0])
#     label = "{}: {:.2f}%".format(label, proba * 100)
#     print(label)
    
#     if args['show']:   
#         # draw the label on the image
#         output = imutils.resize(orig, width=400)
#         cv2.putText(output, label, (10, 25),cv2.FONT_HERSHEY_SIMPLEX,
#             0.7, (0, 255, 0), 2)       
#         # show the output image
#         cv2.imshow("Output", output)
#         cv2.waitKey(0)


# #python predict.py --model traffic_sign.model -i ../2.png -s
# if __name__ == '__main__':
#     # args = args_parse()
#     args = {}
#     args['model']='traffic_sign.model'
#     args['show']=True
#     args['image'] = '/home/kesci/input/traffic_sign8499/traffic-sign/test/00000/00017_00000.png'
#     predict(args)
    

time: 685 µs


In [20]:
# import the necessary packages
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import cv2

norm_size = 32

def args_parse():
# construct the argument parse and parse the arguments
    ap = argparse.ArgumentParser()
    ap.add_argument("-m", "--model", required=True,
        help="path to trained model model")
    ap.add_argument("-i", "--image", required=True,
        help="path to input image")
    ap.add_argument("-s", "--show", action="store_true",
        help="show predict image",default=False)
    args = vars(ap.parse_args())    
    return args

    
def predict(args):
    # load the trained convolutional neural network
    print("[INFO] loading network...")
    model = load_model(args["model"])
    
    #load the image
    image = cv2.imread(args["image"])
    orig = image.copy()
     
    # pre-process the image for classification
    image = cv2.resize(image, (norm_size, norm_size))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
     
    # classify the input image
    result = model.predict(image)[0]
    #print (result.shape)
    proba = np.max(result)
    label = str(np.where(result==proba)[0])
    label = "{}: {:.2f}%".format(label, proba * 100)
    print(label)
    
    if args['show']:   
        # draw the label on the image
        output = imutils.resize(orig, width=400)
        cv2.putText(output, label, (10, 25),cv2.FONT_HERSHEY_SIMPLEX,
            0.7, (0, 255, 0), 2)       
        # show the output image
        # cv2.imshow("Output", output)
        # cv2.waitKey(0)
        plt.imshow(output)


#python predict.py --model traffic_sign.model -i ../2.png -s
if __name__ == '__main__':
    # args = args_parse()
    args = {}
    args['model']='traffic_sign.model'
    args['show']=True
    args['image'] = '/home/kesci/input/traffic_sign8499/traffic-sign/test/00000/00017_00000.png'
    predict(args)

[INFO] loading network...
[0]: 99.95%


<Figure size 432x288 with 1 Axes>

time: 1.45 s
